In [8]:
%reload_ext autoreload
%autoreload 2
import main
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [40, 30]

In [14]:
import fastai as fai
from data_loader import *
import numpy as np
from torch.utils.data import SubsetRandomSampler
from model import *
from model_bn import *
from torch import optim
import dill
import papermill as pm

In [22]:
optimizer = 'Adam'
num_workers=8
maxsize=1000000
batch_size=2048
n_epochs=500
batch_norm = False

In [23]:
indices = np.random.RandomState(seed=42).permutation(maxsize)

val_indices, train_indices = indices[:maxsize//50], indices[maxsize//50:]
dataset='data/train_dataset.h5'

train_dl = fai.basic_data.DataLoader(DatasetFromHdf5(dataset, maxsize=len(train_indices)), 
                        batch_size=batch_size,
                        sampler=SubsetRandomSampler(train_indices),
                         num_workers=num_workers)

val_dl = fai.basic_data.DataLoader(DatasetFromHdf5(dataset, maxsize=len(val_indices)), 
                        batch_size=batch_size, 
                        sampler=SubsetRandomSampler(val_indices),
                         num_workers=num_workers)

db = fai.basic_data.DataBunch(train_dl, val_dl)

In [24]:
input_size = train_dl.dataset.programs.shape[1] + 2*train_dl.dataset.schedules.shape[1] 
output_size = 1

m = None 

if batch_norm:
    m = Model_BN(input_size, output_size)
else:
    m = Model(input_size, output_size)
    
criterion = nn.MSELoss()

l = fai.Learner(db, m, loss_func=criterion)

if optimizer == 'SGD':
    l.opt_func = optim.SGD


In [25]:
l.lr_find()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [22]:
#l.recorder.plot()

In [23]:
lr = 0.01

In [19]:
l.fit_one_cycle(n_epochs, lr)

epoch,train_loss,valid_loss


Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/mohammed/anaconda3/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/home/mohammed/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/mohammed/anaconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/mohammed/anaconda3/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/mohammed/anaconda3/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/home/mohammed/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/mohammed/anaconda3/lib/python3.6/multiprocessing/co

KeyboardInterrupt: 

In [ ]:
l.recorder.plot_losses()

In [36]:
l.save(f"{optimizer}_batch_norm_{batch_norm}")
pm.record("model_name", f"models/{optimizer}_batch_norm_{batch_norm}")
pm.record("losses", list(np.array(l.recorder.losses)))
pm.record("val_losses", list(np.array(l.recorder.val_losses)))

In [38]:
#df = pm.read_notebook('output1.ipynb').dataframe

In [39]:
#df

,name,value,type,filename
0,maxsize,20000,parameter,output1.ipynb
1,n_epochs,1,parameter,output1.ipynb
2,losses,"[4.782155513763428, 4.808725357055664, 7.42696...",record,output1.ipynb
3,model_name,models/Adam_batch_norm_False,record,output1.ipynb
4,val_losses,[3.0963144302368164],record,output1.ipynb
